In [ ]:
!pip install ultralytics
from ultralytics import YOLO
import os
from google.colab import files
from IPython.display import Image, display
import zipfile, glob, shutil

print("Please upload your dataset ZIP file")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(".zip"):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("/content/dataset")
        print(f"Extracted dataset: {filename}")

print("Model Training started")
model = YOLO('yolov8n.pt')

results = model.train(
    data='/content/dataset/data.yaml',
    epochs=50,
    imgsz=640,                          # image size
    batch=16,
    name='vehicle_detection_model',     # name for output folder
    workers=4
)

print("Training completed!")

#TRAINING RESULTS
display(Image(filename='runs/detect/vehicle_detection_model/results.png'))

#IMAGE UPLOAD
print("Upload a test image for detection")
uploaded_test = files.upload()

trained_model = YOLO('runs/detect/vehicle_detection_model/weights/best.pt')

for filename in uploaded_test.keys():
    results = trained_model.predict(source=filename, show=True, conf=0.5)
    print(f"Prediction done for: {filename}")

output_folder = results[0].save_dir
output_files = glob.glob(os.path.join(output_folder, "*.jpg"))

for f in output_files:
    display(Image(filename=f))
for f in output_files:
    shutil.move(f, '/content/')
    files.download(f)

print("All tasks completed successfully")

In [ ]:
print("Downloading the best trained model...")
try:
    files.download('/content/runs/detect/vehicle_detection_model/weights/best.pt')
    print("Downloaded best.pt successfully")
except FileNotFoundError:
    print("Error: best.pt not found. Please ensure the training completed successfully.")
except Exception as e:
    print(f"An error occurred during download: {e}")